In [41]:
import sys
sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt
from dmet.model import hubbard
from pyscf.fci import direct_spin1, fci_slow, FCI, cistring
from pyscf.scf import hf

def trans_energy(h1e, eri, cibra, ciket, norb, nelec, link_index=None):
    '''Compute the FCI electronic energy for given Hamiltonian and FCI vector.
    '''
    h2e = direct_spin1.absorb_h1e(h1e, eri, norb, nelec, .5)
    ci1 = direct_spin1.contract_2e(h2e, ciket, norb, nelec, link_index)
    return np.dot(cibra.reshape(-1), ci1.reshape(-1))


def _get_projected_basis_1site(siteidx, ci0, norb, nelec):
    num_string = cistring.num_strings(norb, nelec//2)
    if ci0 is None: ci0 = np.random.random((num_string, num_string))
    strs = cistring.make_strings(range(norb), nelec//2)

    occidx = np.where(strs & 2**siteidx == 2**siteidx)[0]
    viridx = np.where(strs & 2**siteidx == 0)[0]

    ci_ud = np.zeros_like(ci0)
    ci_ud[np.ix_(occidx, occidx)] += ci0[np.ix_(occidx, occidx)]

    ci_u0 = np.zeros_like(ci0)
    ci_u0[np.ix_(occidx, viridx)] += ci0[np.ix_(occidx, viridx)]

    ci_0d = np.zeros_like(ci0)
    ci_0d[np.ix_(viridx, occidx)] += ci0[np.ix_(viridx, occidx)]

    ci_00 = np.zeros_like(ci0)
    ci_00[np.ix_(viridx, viridx)] += ci0[np.ix_(viridx, viridx)]    

    return ci_ud, ci_u0, ci_0d, ci_00

def calc_char(L, U):

    t = 1

    mol = hubbard.HubbardMole(lx=L, ly=1, t=t, U=U)
    mol.nelectron = L
    mf = hubbard.RHF_SpinHubbard(mol)
    mf.pbc = True
    mf.kernel()

    h1e = mf.get_hcore()
    eri = mf._eri

    e, hop, diag, c = fci_slow.kernel(h1e, eri, L, L)
    c = c.reshape((int(np.sqrt(c.shape[0])), int(np.sqrt(c.shape[0]))))

    print(e)

    wfnlst = [c]
    for i in range(L):
        wfnlst.extend(list(_get_projected_basis_1site(i, c, L, L)))

    exact_H, exact_S = np.zeros((len(wfnlst), len(wfnlst))), np.zeros((len(wfnlst), len(wfnlst)))

    for i, wfni in enumerate(wfnlst):
        for j, wfnj in enumerate(wfnlst):
            exact_H[i,j] += trans_energy(h1e, eri, wfni, wfnj, L, L)
            exact_S[i,j] += np.dot(wfni.reshape(-1), wfnj.reshape(-1))
            
    wS_exact, vS_exact = np.linalg.eig(exact_S)
    mask = wS_exact > 1e-5
    S_proj = vS_exact.T[mask] @ (exact_S+exact_S.T)/2 @ vS_exact[:,mask]
    H_proj = vS_exact.T[mask] @ (exact_H+exact_H.T)/2 @ vS_exact[:,mask]
    wHp_exact, vHp_exact = hf.eig(H_proj, S_proj)   # solve in truncated area

    print("Energy for exact-TN = ", wHp_exact)
    coeff_in_basis = vS_exact[:,mask] @ vHp_exact
    print(coeff_in_basis.shape)
    wfnbasis = np.array(wfnlst)

    def calc_trans(ci0, norb, nelec):
        neleca, nelecb = nelec
        threshold = 2**norb

        str_a = list(cistring.make_strings(range(norb), neleca))
        str_b = list(cistring.make_strings(range(norb), nelecb))
        res = 0

        for i, stra in enumerate(str_a):
            changed_stra = 2*stra
            if changed_stra >= threshold: changed_stra = changed_stra%threshold+1
            changed_i = str_a.index(changed_stra)

            for j, strb in enumerate(str_b):
                changed_strb = 2*strb
                if changed_strb >= threshold: changed_strb = changed_strb%threshold+1
                changed_j = str_b.index(changed_strb)

                res += ci0[i,j] * ci0[changed_i, changed_j]

        return res

    char = []

    wfns = np.einsum("tij,tk->kij", wfnlst, coeff_in_basis)
    for i in range(wfns.shape[0]):
        ci = wfns[i]
        character = calc_trans(ci, L, (L//2,L//2)).real
        print(character)
        char.append(character)

    np.savetxt("./results/energy-exactTN-{}-{}.txt".format(L,U), wHp_exact)
    np.savetxt("./results/char-exactTN-{}-{}.txt".format(L, U), np.array(char))

for U in [6,]:
   calc_char(L=6, U=U)

converged SCF energy = 0.999999999999998


Initialize <dmet.model.hubbard.HubbardMole object at 0x7fbada92f5e0> in <dmet.model.hubbard.RHF_SpinHubbard object at 0x7fbada92f250>


-2.648517564261159
Energy for exact-TN =  [-2.64851756 -2.00357946 -1.61373873 -1.61373872 -1.33850391 -1.3385039
  2.68948744  2.6894875   3.96915838  3.96915838  4.31183694  4.35268012
  4.75735949  4.93631265  4.93631266  5.01579411  5.01579415]
(25, 17)
1.0000000000000002
-0.999999999999998
0.4999999999999986
0.49999999999999917
-0.49999999999999895
-0.5000000000000001
0.4999999999999997
0.49999999999999073
-0.4999999999999987
-0.4999999999999984
-0.9999999999994595
0.9999999999994594
-0.9999999999999729
0.4999999999999991
0.49999999999998496
-0.49999999999999983
-0.4999999999999937


In [12]:
t = 1
L = 6
U = 6

mol = hubbard.HubbardMole(lx=L, ly=1, t=t, U=U)
mol.nelectron = L
mf = hubbard.RHF_SpinHubbard(mol)
mf.pbc = True
mf.kernel()
h1e = mf.get_hcore()
eri = mf._eri
e, hop, diag, c = fci_slow.kernel(h1e, eri, L, L)
c = c.reshape((int(np.sqrt(c.shape[0])), int(np.sqrt(c.shape[0]))))
print(e)
wfnlst = [c]
for i in range(L):
    wfnlst.extend(list(_get_projected_basis_1site(i, c, L, L)))
exact_H_fci, exact_S_fci = np.zeros((len(wfnlst), len(wfnlst))), np.zeros((len(wfnlst), len(wfnlst)))

for i, wfni in enumerate(wfnlst):
    for j, wfnj in enumerate(wfnlst):
        exact_H_fci[i,j] += trans_energy(h1e, eri, wfni, wfnj, L, L)
        exact_S_fci[i,j] += np.dot(wfni.reshape(-1), wfnj.reshape(-1))

converged SCF energy = 0.999999999999998


Initialize <dmet.model.hubbard.HubbardMole object at 0x7fba7bcd33d0> in <dmet.model.hubbard.RHF_SpinHubbard object at 0x7fba7bcd3280>


-2.648517564261159


In [44]:
#Block2
import numpy as np
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock3.block2.io import MPSTools, MPOTools
from pyblock3.algebra.core import SparseTensor

L = 6
N = 6
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

t = 1
U = 6

b = driver.expr_builder()

# hopping term
b.add_term("cd",
    np.array([[[i, (i+1)%L], [(i+1)%L, i]] for i in range(L)]).flatten(),
    [-t] * 2 * (L))
b.add_term("CD",
    np.array([[[i, (i+1)%L], [(i+1)%L, i]] for i in range(L)]).flatten(),
    [-t] * 2 * (L))

# onsite term
b.add_term("cdCD",
    np.array([[i, ] * 4 for i in range(L)]).flatten(),
    [U] * L)

mpo = driver.get_mpo(b.finalize(), iprint=2, add_ident=False)
ket = driver.get_random_mps(tag="KET", bond_dim=400, nroots=1)

def run_dmrg(driver, mpo):
    bond_dims = [200] * 4 + [400] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(
        mpo,
        ket,
        n_sweeps=25,
        bond_dims=bond_dims,
        noises=noises,
        thrds=thrds,
        iprint=2,
    )

energies = run_dmrg(driver, mpo)
print(energies)



Build MPO | Nsites =     6 | Nterms =         30 | Algorithm = FastBIP | Cutoff = 1.00e-14
 Site =     0 /     6 .. Mmpo =     6 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /     6 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.7500 Tmvc = 0.000 T = 0.003
 Site =     2 /     6 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8500 Tmvc = 0.000 T = 0.003
 Site =     3 /     6 .. Mmpo =    10 DW = 0.00e+00 NNZ =       15 SPT = 0.8500 Tmvc = 0.000 T = 0.001
 Site =     4 /     6 .. Mmpo =     6 DW = 0.00e+00 NNZ =       15 SPT = 0.7500 Tmvc = 0.000 T = 0.001
 Site =     5 /     6 .. Mmpo =     1 DW = 0.00e+00 NNZ =        6 SPT = 0.0000 Tmvc = 0.000 T = 0.005
Ttotal =      0.014 Tmvc-total = 0.000 MPO bond dimension =    10 MaxDW = 0.00e+00
NNZ =           72 SIZE =          332 SPT = 0.7831

Rank =     0 Ttotal =      0.034 MPO method = FastBipartite bond dimension =      10 NNZ =           72 SIZE =          332 SPT = 0.7831
Environment initi

In [45]:
def calc_trans_mps(mps, mpsr, nelec=None):

    t = mps.tensors
    tr = mpsr.tensors
    norb = len(t)
    if nelec is None: nelec = norb

    res = SparseTensor._tensordot(t[0], t[1], axes=([2],[0]))
    res = SparseTensor._tensordot(res, tr[0], axes=([2],[1]))
    res = SparseTensor._tensordot(res, t[2], axes=([2],[0]))
    res = SparseTensor._tensordot(res, tr[1], axes=([3,4],[0,1]))
    for i in range(2, norb-1):
        res = SparseTensor._tensordot(res, t[i+1], axes=([3],[0]))
        res = SparseTensor._tensordot(res, tr[i], axes=([3,4],[0,1]))

    # Phase Factor
    t2 = mpsr.copy().tensors
    t2[norb-1].blocks[1] *= -1**(nelec-1)
    t2[norb-1].blocks[2] *= -1**(nelec-1)

    res = SparseTensor._tensordot(res, t2[norb-1], axes=([4,1,3],[0,1,2]))
    return res.blocks[0].reshape(-1)[0]


In [46]:
mps = driver.adjust_mps(ket, dot=1)[0]
pyket = MPSTools.from_block2(mps)

from pyblock3.algebra.core import SubTensor

wfnlst = [pyket]

pympo = MPOTools.from_block2(mpo.prim_mpo)

qnlst = ((2,0), (1,1), (1,-1), (0,0))

for i in range(L):
    for qn in qnlst:
        mps2 = pyket.copy()
        for j, block in enumerate(mps2.tensors[i].blocks):
            ql = block.q_labels
            if ql[1].n != qn[0] or ql[1].twos != qn[1]:
                zeroblock = SubTensor(np.zeros_like(np.asarray(block)), q_labels=ql)
                mps2.tensors[i].blocks[j] = zeroblock
                #print(i, j, mps2.tensors[i].blocks[j])
                #print(i, j, pyket.tensors[i].blocks[j])
        wfnlst.append(mps2)



In [49]:
exact_H = np.zeros((len(wfnlst), len(wfnlst)))
exact_S = np.zeros((len(wfnlst), len(wfnlst)))

for i, wfni in enumerate(wfnlst):
    for j, wfnj in enumerate(wfnlst):
        if j >= i:
            print(i, j)
            frac = 1.
            if i == j: frac = .5
            exact_S[i,j] += (wfni @ wfnj) * frac
            exact_H[i,j] += (wfni @ (pympo @ wfnj)) * frac

exact_H = (exact_H + exact_H.T)
exact_S = (exact_S + exact_S.T)

np.savetxt("exact-S2.txt", exact_S)
np.savetxt("exact-H2.txt", exact_H)


0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
4 22
4 23
4 24
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
5 22
5 23
5 24
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
6 22
6 23
6 24
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
7 17
7 18
7 19
7 20
7 21
7 22
7 23
7 24
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
8 16
8 17
8 18
8 19
8 20
8 21
8 22
8 23
8 24
9 9
9 10
9 11
9 12
9 13
9 14
9 15
9 16
9 17
9 18
9 19
9 20
9 21
9 22
9 23
9 24
10 10
10 11
10 12
10 13
10 14


In [48]:
from pyscf.scf import hf
wS_exact, vS_exact = np.linalg.eig(exact_S)
mask = wS_exact > 1e-5
S_proj = vS_exact.T[mask] @ (exact_S+exact_S.T)/2 @ vS_exact[:,mask]
print(np.diag(S_proj))

H_proj = vS_exact.T[mask] @ (exact_H+exact_H.T)/2 @ vS_exact[:,mask]

wHp_exact, vHp_exact = hf.eig(H_proj, S_proj)   # solve in truncated area

print("Energy for exact-TN = ", wHp_exact)
coeff_in_basis = vS_exact[:,mask] @ vHp_exact
print(coeff_in_basis.shape)

[3.38850419+0.j 1.27694516+0.j 0.47149307+0.j 0.47149483+0.j
 0.155928  +0.j 0.19896819+0.j 0.19896755+0.j 0.11832024+0.j
 0.15183075+0.j 0.15183063+0.j 0.02260804+0.j 0.0933353 +0.j
 0.09333538+0.j 0.06464916+0.j 0.06464922+0.j 0.03857015+0.j
 0.03857012+0.j]
Energy for exact-TN =  [-2.64851756 -2.00357957 -1.61373945 -1.61373885 -1.3385047  -1.33850382
  2.6894887   2.68948928  3.96915873  3.96915898  4.311837    4.35268138
  4.75735949  4.93631258  4.93631335  5.01579391  5.01579457]
(25, 17)


[-2.64851756 -2.00357946 -1.61373873 -1.61373872 -1.33850391 -1.3385039
  2.68948744  2.6894875   3.96915838  3.96915838  4.31183694  4.35268012
  4.75735949  4.93631265  4.93631266  5.01579411  5.01579415]

In [43]:
for i in range(coeff_in_basis.shape[1]):
    coeff = coeff_in_basis[:,i]
    themps = 0
    for j in range(coeff_in_basis.shape[0]):
        themps += coeff[j] * wfnlst[j]
    print(calc_trans_mps(themps, themps).real)

0.9999999999979433
-0.9999999999969788
0.4999999999965943
0.4999999999812091
-0.49999999999927464
-0.4999999999926248
0.49999999999517575
0.49999999999729094
-0.4999999999968997
-0.4999999999972976
-0.9999999999928912
0.999999999986469
-0.9999999999807683
0.4999999999916558
0.49999999998912953
-0.4999999999948325
-0.49999999999045597


In [58]:
exact_H2 = np.loadtxt("./exact-H2.txt")
exact_H = np.loadtxt("./exact-H.txt")

np.linalg.norm(exact_H2 - exact_H)

1.8620739875960772e-05